In [22]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [23]:
from static_grader import grader

# MapReduce


## Introduction


We are going to be running MapReduce jobs on the Wikipedia data set.  The data set is available  on S3: `s3://dataincubator-course/mrdata/simple/`.  It may be downloaded with the `aws s3 sync` command or via HTTPS from `https://s3.amazonaws.com/dataincubator-course/mrdata/simple/part-000*`.

For development, you can even use a single chunk (eg. part-00026.xml.bz2). That is small enough that mrjob can process the chunk in a few seconds. Your development cycle should be:

1.  Get your job to work locally on one chunk.  This will greatly speed up your
development.  To run on local:
```bash
python job_file.py -r local data/wikipedia/simple/part-00026.xml.bz2 > /tmp/output.txt
```
    
2.  Get your job to work on the full data set on GCP (Google Cloud Platform).  This will greatly speed up your production.  To run on GCP ([details](https://pythonhosted.org/mrjob/guides/dataproc-quickstart.html)):
```bash
python job_file.py -r dataproc data/wikipedia/simple/part-00026.xml.bz2 \
    --output-dir=gs://my-bucket/output/ \
    --no-output 
```

    Not that you can also pass an entire local directory of data (eg. `data/simple/`) as the input.


In [2]:
!aws s3 sync s3://dataincubator-course/mrdata/simple/ . --exclude "*" --include "part-000*"

download: s3://dataincubator-course/mrdata/simple/part-00006.xml.bz2 to ./part-00006.xml.bz2
download: s3://dataincubator-course/mrdata/simple/part-00005.xml.bz2 to ./part-00005.xml.bz2
download: s3://dataincubator-course/mrdata/simple/part-00004.xml.bz2 to ./part-00004.xml.bz2
download: s3://dataincubator-course/mrdata/simple/part-00007.xml.bz2 to ./part-00007.xml.bz2
download: s3://dataincubator-course/mrdata/simple/part-00008.xml.bz2 to ./part-00008.xml.bz2
download: s3://dataincubator-course/mrdata/simple/part-00002.xml.bz2 to ./part-00002.xml.bz2
download: s3://dataincubator-course/mrdata/simple/part-00001.xml.bz2 to ./part-00001.xml.bz2
download: s3://dataincubator-course/mrdata/simple/part-00014.xml.bz2 to ./part-00014.xml.bz2
download: s3://dataincubator-course/mrdata/simple/part-00015.xml.bz2 to ./part-00015.xml.bz2
download: s3://dataincubator-course/mrdata/simple/part-00013.xml.bz2 to ./part-00013.xml.bz2
download: s3://dataincubator-course/mrdata/simple/part-00010.xml.bz2 t

In [ ]:
python my_job.py -r local ./part-00026.xml.bz2 > /tmp/output.txt

### Note on Memory

There's a large difference between developing locally on one chunk and running your job on the entire data set.  While you can get away with sloppy memory use locally, you really need to keep memory usage down if you hope to be able to complete the miniproject.  Remember, memory needs to be $O(1)$, not $O(n)$, in input.


### Multiple MapReduces

You can combine multiple steps by overriding the [steps method](https://pythonhosted.org/mrjob/guides/writing-mrjobs.html#multi-step-jobs).  Usually your MapReduce might look like this
```python
from mrjob.job import MRJob

class SingleMRJob(MRJob):
    def mapper(self, key, value):
        pass

    def reducer(self, key, values):
        pass
```

`MRJob` automatically uses the `mapper` and `reducer` methods.  To specify multiple steps, you need to override the `steps` method:

```python
from mrjob.job import MRJob
from mrjob.step import MRStep

class MultipleMRJob(MRJob):
    def mapper1(self, key, value):
        pass

    def reducer1(self, key, values):
        pass
        
    def mapper2(self, key, value):
        pass

    def reducer2(self, key, values):
        pass
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper1, reducer=self.reducer1),
            MRStep(mapper=self.mapper2, reducer=self.reducer2),
        ]
```

As a matter of good style, we recommend that you actually write each individual MapReduce as its own class.  Then write a wrapper module whose sole job is to combine those MapReduces by overriding `steps`.

Some simple boilerplate for this, taking advantage of the default `steps` function that we get for free in a single-step `MRJob` class:

```python
class FirstStep(MRJob):
  def mapper(self, key, value):
    pass
  def reducer(self, key, values):
    pass
  
class SecondStep(MRJob):
  def mapper(self, key, value):
    pass
  def reducer(self, key, values):
    pass
  
class SteppedJob(MRJob):
  """
  A two-step job that first runs FirstStep's MR and then SecondStep's MR
  """
  def steps(self):
    return FirstStep().steps() + SecondStep().steps()
```



### Note on Style

Here are some helpful articles on how mrjob works and how to pass parameters to your script:
  - [How mrjob is run](https://pythonhosted.org/mrjob/guides/concepts.html#how-your-program-is-run)
  - [Adding pass-through options](https://pythonhosted.org/mrjob/job.html#mrjob.job.MRJob.add_passthrough_option)
  - [An example of someone solving similar problems](http://arunxjacob.blogspot.com/2013/11/hadoop-streaming-with-mrjob.html)

See the notebook "MR_Mrjob_in_the_Cloud" in the datacourse for more details on running on external services.

Finally, if you are find yourself processing a lot of special cases, you are probably doing it wrong.  For example, MapReduce jobs for `Top100WordsSimpleWikipediaPlain`, `Top100WordsSimpleWikipediaText`, and `Top100WordsSimpleWikipediaNoMetaData` are less than 150 lines of code (including generous blank lines and boilerplate code).  

## Question 1: top100_words_simple_plain

Return a list of the top 100 words in an article text (in no particular order). You will need to write this as two MapReduce steps:

1. The first job is similar to standard word count but with a few tweaks. The data provided for Wikipedia is in `*.xml.bz2` format.  `MRJob` will automatically decompress `bz2`.  We'll deal with the XML in the next question. For now, just treat it as text.  A few hints:
   - To split the words, use the regular expression "\w+".
   - Words should be treated in a non case-sensitive fashion: i.e. "The" and "the" reference to the same word.  You can use `string.lower()` to get a single case-insensitive canonical version of the data.

2. The second job will take a collection of pairs `(word, count)` and filter for only the highest 100.  A few notes:
    - **Passing parameters:** To make the code more reusable, make it find the largest `n` words where `n` is a parameter obtained via [`jobconf_from_env`](https://pythonhosted.org/mrjob/utils-compat.html).
    - **Keeping track of the top n:** We have to keep track of the `n` most popular words.  As long as `n` is small, e.g. 100, we can keep track of the *running largest n* in memory with a priority queue. We suggest taking a look at `heapq` ([details](https://docs.python.org/2/library/heapq.html)), part of the Python standard library for this.  It allows you to push elements into a list while keeping track of the highest priority element.
```python
h = []
heappush(h, (5, 'write code'))
heappush(h, (7, 'release product'))
heappush(h, (1, 'write spec'))
heappush(h, (3, 'create tests'))
heappop(h)  // returns (1, 'write spec')
```
   
       A naive implementation would cost $O(1)$ to insert but $O(n)$ to retrieve.  `heapq` uses a [self-balancing binary search tree](https://en.wikipedia.org/wiki/Self-balancing_binary_search_tree) to enable $O(\log(n))$ insertion and $O(1)$ removal. You may be asked about this data structure during an interview, so it is good to get practice with it now.
    - **Working across nodes:** To obtain the largest `n`, we need to first obtain the largest elements per chunk from the mapper, output them to the same key (reducer), and then collect the largest n elements of those in the reducer. (**Question:** why does this guarantee that we have found the largest n over the entire set?)
    - **Working within a node:** Given that we are using a priority queue, we will need to first initialize it, then `push` or `pushpop` each record to it, and finally output the top `n` after seeing each record.  For mappers, notice that these three phases correspond nicely to these three functions:
        - `mapper_init`
        - `mapper`
        - `mapper_final`

    There are similar functions in the reducer.  Also, while the run method to launch the MapReduce job is a class method:
```python
        if __name__ == '__main__':
            MRWordCount.run()
```
     actual instances of our MapReduce are instantiated on the map and reduce nodes.  More precisely, a separate mapper class is instantiated in each map node and a reducer class is instantiated in each reducer node.  This means that the three mapper functions can pass state through `self`, e.g. `self.heap`. Remember that to pass state between the map and reduce phase, you will have to use `yield` in the mapper and read each line in the reducer. (**Question:** Can you pass state between two mappers?)

**Checkpoint:**
- Total unique words: 1,584,646

In [ ]:
#python my_job.py -r local ./data/part-0000*.xml.bz2 > ./tmp/output.txt
python my_job.py -r local ./data/part-000*.xml.bz2 > ./tmp/output.txt | More

In [60]:
import os
directory =os.path.join("./MrJob/tmp") 
f=open(os.path.join(directory, 'output.txt'),'r')
d={}
s=0
for line in f.readlines():
    line=line.rstrip('"\n')
    d[line.split('\t"',1)[1]]=int(line.split('\t"',1)[0])
words_sorted = sorted(d.items(), key=lambda kv: kv[1], reverse=True)
len(words_sorted)  
#words_sorted[0:100]
#    for i in re.split('\t',line):

2520

In [58]:
import os
directory =os.path.join("./MrJob/tmp")
d={}
s=0
for i in range(27):
    f=open(os.path.join(directory, 'output'+str("{0:0=2d}".format(i))+'.txt'),'r')
    for line in f.readlines():
        line=line.rstrip('"\n')
        if line.split('\t"',1)[1] in d.keys():
            d[line.split('\t"',1)[1]]=d[line.split('\t"',1)[1]]+int(line.split('\t"',1)[0])
        else:
            d[line.split('\t"',1)[1]]=int(line.split('\t"',1)[0])
words_sorted = sorted(d.items(), key=lambda kv: kv[1], reverse=True)
len(words_sorted)  
#words_sorted[0:100]
#    for i in re.split('\t',line):

1871079

In [57]:
'one' in d.keys()

True

In [61]:
import re
re.compile(r"\w+").findall('sas asas asas) the . sasa112 1212  the sasa a asa')

['sas', 'asas', 'asas', 'the', 'sasa112', '1212', 'the', 'sasa', 'a', 'asa']

In [59]:
def top100_words_simple_plain():
    return words_sorted[0:100] 
#    return [("the", 1586419)] * 100

grader.score(question_name='mr__top100_words_simple_plain', func=top100_words_simple_plain)

Your score:  0.9500000000000006


## Question 2: top100_words_simple_text

Notice that the words "page" and "text" make it into the top 100 words in the previous problem.  These are not common English words!  If you look at the XML formatting, you'll realize that these are XML tags.  You should parse the files so that tags like `<page></page>` should not be included in your total, nor should words outside of the tag `<text></text>`.

**Hints**:
1. Both `xml.etree.elementtree` from the Python standard library or `lxml.etree` parse XML. `lxml` is significantly faster though and avoids some bugs.

2. In order to parse the text, we will have to accumulate a `<page></page>` worth of data and then split the resulting string into words.

3. Don't forget that the Wikipedia format can have multiple revisions but you only want the latest one.

4. What happens if content from a page is split across two different mappers? How does this problem scale with data size?

**Checkpoint:**
- Total unique words: 867,871

In [71]:
import lxml.etree
root = etree.parse(r'./MrJob/data/part-00000.xml.bz2')
et = lxml.etree.fromstring('''<item><img src="cat.jpg" /> Picture of a cat</item>''')
et.find('img').tail

SyntaxError: invalid or missing encoding declaration for './MrJob/data/part-00000.xml.bz2' (<string>)

In [ ]:
python q2_4.py -r local ./data/text_all.txt  > ./tmp/output2.txt

In [175]:
import os
directory =os.path.join("./MrJob/tmp")
d={}
s=0
for i in range(1):
    f=open(os.path.join(directory, 'output'+str(2)+'.txt'),'r')
    for line in f.readlines():
        line=line.lstrip('"').rstrip('\n')
        line=line.split('"\t')
        if line[0] in d.keys():
            d[line[0]]=d[line[0]]+int(line[1])
        else:
            d[line[0]]=int(line[1])
words_sorted = sorted(d.items(), key=lambda kv: kv[1], reverse=True)
len(words_sorted)
#words_sorted[:100]

931584

In [ ]:
import mwparserfromhell
from mwparserfromhell import parser
value="{{jjb}}{{g}}"
wikicode = mwparserfromhell.parse(value)
bigString = wikicode.strip_code()
bigString

In [176]:
def top100_words_simple_text():
    return words_sorted[:100]

grader.score(question_name='mr__top100_words_simple_text', func=top100_words_simple_text)

Your score:  0.9700000000000006


## Question 3: top100_words_simple_no_metadata


Finally, notice that "www" and "http" make it into the list of top 100 words in the previous problem.  These are also not common English words either!  These are clearly from the URL in hyperlinks.  Looking at the format of [Wikipedia links](http://en.wikipedia.org/wiki/Help:Wiki_markup#Links_and_URLs) and [citations](http://en.wikipedia.org/wiki/Help:Wiki_markup#References_and_citing_sources), you'll notice that they tend to appear within single and double brackets and curly braces.

**Hint**:
You can either write a simple parser to eliminate the URLs within brackets, angle braces, and curly braces or you can use a package like the colorfully-named [`mwparserfromhell`](https://github.com/earwig/mwparserfromhell/), which has been provisioned on `mrjob` and supports the convenient helper function `strip_code()` (which is used by the reference solution).

**Checkpoint:**
- Total unique words: 618,410

In [181]:
import os
directory =os.path.join("./MrJob/tmp")
d={}
s=0
for i in range(1):
    f=open(os.path.join(directory, 'output'+str(3)+'.txt'),'r')
    for line in f.readlines():
        line=line.lstrip('"').rstrip('\n')
        line=line.split('"\t')
        if line[0] in d.keys():
            d[line[0]]=d[line[0]]+int(line[1])
        else:
            d[line[0]]=int(line[1])
words_sorted = sorted(d.items(), key=lambda kv: kv[1], reverse=True)
len(words_sorted)

words_sorted[:100]

[('n', 3535544),
 ('the', 1382504),
 ('of', 746958),
 ('in', 567163),
 ('and', 547192),
 ('a', 507283),
 ('to', 416224),
 ('is', 390754),
 ('ncategory', 265771),
 ('was', 209422),
 ('xe2', 202801),
 ('it', 201328),
 ('x80', 200970),
 ('for', 183357),
 ('0', 159950),
 ('on', 159391),
 ('that', 154241),
 ('as', 146911),
 ('s', 142411),
 ('align', 141125),
 ('xc3', 136109),
 ('by', 131593),
 ('are', 128969),
 ('from', 125740),
 ('1', 125434),
 ('2', 124002),
 ('with', 120777),
 ('he', 118543),
 ('this', 102596),
 ('be', 101229),
 ('i', 96700),
 ('or', 94969),
 ('at', 93111),
 ('an', 92607),
 ('center', 91638),
 ('x93', 87082),
 ('not', 86622),
 ('people', 78466),
 ('style', 76501),
 ('3', 75881),
 ('other', 75720),
 ('they', 74332),
 ('his', 72038),
 ('xd0', 70341),
 ('t', 67522),
 ('have', 66919),
 ('utc', 65754),
 ('has', 63386),
 ('also', 60330),
 ('american', 59394),
 ('bgcolor', 59167),
 ('one', 58041),
 ('4', 57329),
 ('right', 56780),
 ('talk', 56350),
 ('which', 54441),
 ('can', 5

In [179]:
def parse_xml(path):
   parsedfile = path.replace('.xml', '.txt')
   cnone = 0
   textlist = []
   with open(parsedfile, 'w',encoding='utf-8') as parsed:
       with open(path) as xml_file:
           root = etree.fromstring(xml_file.read())
           nodelist = root.findall('.//text')
           for node in nodelist:
               textlist.append(node.text)

           for text in textlist:
               if text:
                   parsed.write(cleanup(text)+' ')

In [182]:
def top100_words_simple_no_metadata():
    return words_sorted[:100]

grader.score(question_name='mr__top100_words_simple_no_metadata', func=top100_words_simple_no_metadata)

Your score:  0.9000000000000006


In [133]:
a='प्रिथिबी'
a.encode('utf-8')

b'\xe0\xa4\xaa\xe0\xa5\x8d\xe0\xa4\xb0\xe0\xa4\xbf\xe0\xa4\xa5\xe0\xa4\xbf\xe0\xa4\xac\xe0\xa5\x80'

## Question 4: link_stats_simple

Let's look at some summary statistics on the number of unique links on a page to other Wikipedia articles.  Return the number of articles (count), average number of links, standard deviation, and the 25%, median, and 75% quantiles.

1. Notice that the library `mwparserfromhell` supports the method `filter_wikilinks()`.
2. You will need to compute these statistics in a way that requires $O(1)$ memory.  You should be able to compute the first few (i.e. non-quantile) statistics exactly by looking at the first few moments of a distribution. The quantile quantities can be accurately estimated by using "reservoir sampling" with a large reservoir.
3. If there are multiple links to the article, have it only count for 1.  This keeps our results from becoming too skewed.
4. Don't forget that some (a surprisingly large number of) links have Unicode! Make sure you treat them correctly.

In [194]:


#text_file = open("text_all_links.txt", "w",encoding='utf-8')#, encoding='utf-8')

textlist=[]
with open(r'./fixed_part-00000-Copy1.xml') as xml_file:
    root = etree.fromstring(xml_file.read())
    nodelist = root.findall('.//text')
    for node in nodelist:
        if node.text:
            wikicode = mwparserfromhell.parse(node.text)
            links = wikicode.filter_wikilinks()

#            for link in links:
#                text_file.write(link.text+' ')
#text_file.close()

In [355]:
import sys

directory =os.path.join("./MrJob/data")
text_file = open("link_stat2.txt", "w",encoding='utf-8')#, encoding='utf-8')
s=0
for i in range(27):
#    fd=open(r'./part-000'+str("{0:0=2d}".format(i))+'.xml')#, encoding='utf-8')
    textlist=[]
#    with open(os.path.join(directory, r'./fixed_part-000'+str("{0:0=2d}".format(i))+'.xml'),'r') as xml_file:
    with open(os.path.join(directory, r'./part-000'+str("{0:0=2d}".format(i))+'.xml'),'r') as xml_file:
        root = etree.fromstring('<a>\n'+xml_file.read()+'\n</a>')
        pagelist = root.findall('.//page')
        for page in pagelist:
            nodelist = page.findall('.//text')
#            if len(nodelist)>1: print('hi')
            for node in nodelist:
                if node.text:
                    s+=1
                    wikicode = mwparserfromhell.parse(node.text)
                    links = wikicode.filter_wikilinks()
                    links_str=[]
                    for i in links: links_str.append(str(i))
                    links_str=remove_dupe(links_str)
#                    if len(remove_dupe(links_str))!=len(links): 
#                        print(len(remove_dupe(links_str))-len(links))
#                        sys.exit("Error message")
#                    links = list(set(str(links)))
                    text_file.write(str(s)+' '+str(len(links_str))+'\n')
#                else: print(page.text)
        #            for link in links:
        #                text_file.write(str(s)+' '+str(len(links)))
text_file.close()
def remove_dupe(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [358]:
from statistics import mean,stdev,median
import os
import numpy as np
#directory =os.path.join("./MrJob/data")
directory =os.path.join("./")
d={}
s=0
f=open(os.path.join(directory, 'link_stat2.txt'),'r')
for line in f.readlines():
    s+=1
    line=line.lstrip('"').rstrip('\n')
    line=line.split(' ')
#    if line[0] in d.keys():
#        d[line[0]]=d[line[0]]+int(line[1])
#    else:
#        d[line[0]]=int(line[1])
    d[line[0]]=int(line[1])
f.close()
words_sorted = sorted(d.items(), key=lambda kv: kv[1])#, reverse=True)
pages_count=len(words_sorted)
links_count=sum(d.values())
links_mean=mean(d.values())
links_stdev=stdev(d.values())
links_median=median(d.values())
links_75p =np.percentile(np.array(list(d.values())), 75)
links_25p =np.percentile(np.array(list(d.values())), 25)

#words_sorted[:100]

In [360]:
pages_count

188408

In [362]:

def link_stats_simple():

    return [
        ("count", int(pages_count)),
        ("mean", links_mean),
        ("stdev",links_stdev),
        ("25%", int(links_25p)),
        ("median", int(links_median)),
        ("75%", int(links_75p)),
    ]

grader.score(question_name='mr__link_stats_simple', func=link_stats_simple)

Your score:  0.9999999999999999


*Copyright &copy; 2018 The Data Incubator.  All rights reserved.*